归一化：目的是对节点的特征向量做行L1归一化，每一行的和是1，具体实现是创建了一个每一行和的倒数的对角矩阵乘以特征向量（和度的-1乘X获得邻居求和的平均值同理）

In [1]:
import scipy.sparse as sp
data = [1, 1, 2]
row = [0, 1, 1]
col = [0, 1, 2]
matrix = sp.coo_matrix((data, (row, col)), shape=(3, 3))
matrix.todense()

matrix([[1, 0, 0],
        [0, 1, 2],
        [0, 0, 0]])

In [2]:
import numpy as np
rowsum  = np.array(matrix.sum(1))
rowsum

array([[1],
       [3],
       [0]], dtype=int64)

np.power cannot be used in float，change it to np.float_power

In [3]:
r_inv = np.float_power(rowsum, -1).flatten()
r_inv[np.isinf(r_inv)] = 0.
r_inv

/home/lzq/anaconda3/envs/tf-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in float_power
  """Entry point for launching an IPython kernel.


array([1.        , 0.33333333, 0.        ])

In [4]:
r_mat_inv = sp.diags(r_inv)
r_mat_inv.toarray()

array([[1.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.        ],
       [0.        , 0.        , 0.        ]])

In [5]:
features = r_mat_inv.dot(matrix)
features.toarray()

array([[1.        , 0.        , 0.        ],
       [0.        , 0.33333333, 0.66666667],
       [0.        , 0.        , 0.        ]])

csr_matrix:data是矩阵的非零值，indices是和非零值一一对应的所在行的列位置，indptr是总计非零值的个数，第一个元素默认是0，从第二个元素开始记录每行非零的值个数，这个再结合data按照顺序就可以确定一个稀疏矩阵

In [6]:
indptr = np.array([0, 2, 4, 6])
indices = np.array([0, 2, 1, 2, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
matrix = sp.csr_matrix((data, indices, indptr), shape=(3, 3))
matrix.todense()

matrix([[1, 0, 2],
        [0, 3, 4],
        [0, 5, 6]])

lil_matrix:可以切片，更新数据，非常灵活。

In [7]:
data = [1, 1, 2]
row = [0, 1, 1]
col = [0, 1, 2]
matrix = sp.coo_matrix((data, (row, col)), shape=(3, 3))
matrix.todense()


matrix([[1, 0, 0],
        [0, 1, 2],
        [0, 0, 0]])

In [8]:
matrix = matrix.tolil()
matrix[0,2] = matrix[1,1]
matrix.todense()

matrix([[1, 0, 1],
        [0, 1, 2],
        [0, 0, 0]], dtype=int64)

In [9]:
matrix[[0,2]] = matrix[1]
matrix.todense()

/home/lzq/anaconda3/envs/tf-cpu/lib/python3.6/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/lzq/anaconda3/envs/tf-cpu/lib/python3.6/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):
/home/lzq/anaconda3/envs/tf-cpu/lib/python3.6/site-packages/scipy/sparse/lil.py:518: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not x.flags.writeable:


matrix([[0, 1, 2],
        [0, 1, 2],
        [0, 1, 2]], dtype=int64)

tf.sparse_placeholder:稀疏占位符,和coo_matrix配合使用,例子如下

In [10]:
row = np.array([0, 0, 1, 3])  # 第几行
col = np.array([0, 2, 1, 3])  # 第几列
data = np.array([4, 9, 7, 5])  # 值
tmp = sp.coo_matrix((data, (row, col)), shape=(4, 4))
tmp.todense()

matrix([[4, 0, 9, 0],
        [0, 7, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 5]])

从TensorFlow 的名字就可以看出张量( tensor ）是一个很重要的概念。在TensorFlow
程序中，所有的数据都通过张量的形式来表示。从功能的角度上看，张量可以被简单理解
为多维数组。其中零阶张量表示标量（ scalar ） ，也就是一个数① ： 第一阶张量为向量C vector),
也就是一个一维数组；第n 阶张量可以理解为一个n 维数组。但张量在Tensor Flow 中的实
现并不是直接采用数组的形式，它只是对TensorFlow 中运算结果的引用。在张量中并没有
真正保存数字，它保存的是如何得到这些数字的计算过程。还是以向量加法为例，当运行
如下代码时，并不会得到加法的结果，而会得到对结果的一个引用

In [12]:
import tensorflow as tf
# tf.constant 是一个计算，这个计算的结果为一个张量， 保存在变量a中。
a= tf . constant([1.0, 2.0] , name ="a")
b = tf . constant ( [2.0 , 3.0] , name ="b")
result = tf.add (a , b , name="add")
print(result)

Tensor("add:0", shape=(2,), dtype=float32)


In [13]:
import tensorflow as tf
x = tf.sparse_placeholder(tf.float32)
with tf.Session() as sess:
    indices = np.mat([tmp.tocoo().row, tmp.tocoo().col]).transpose()
    values = tmp.tocoo().data
    shape = tmp.tocoo().shape
    # feed_dict传入三元组(坐标，非零，维度)
    sp_ten = sess.run(x, feed_dict={x: (indices, values, shape)})
    print("-----------tf.sparse_placeholder效果")
    print(sp_ten)
    dense_tensor = tf.sparse_tensor_to_dense(sp_ten)
    print("-----------tf.sparse_placeholder转化为稠密矩阵")
    print(sess.run(dense_tensor))

-----------tf.sparse_placeholder效果
SparseTensorValue(indices=array([[0, 0],
       [0, 2],
       [1, 1],
       [3, 3]]), values=array([4., 9., 7., 5.], dtype=float32), dense_shape=array([4, 4]))
-----------tf.sparse_placeholder转化为稠密矩阵
[[4. 0. 9. 0.]
 [0. 7. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 5.]]


coo_matrix转化为三元组格式可以直接传入tf.sparse_placeholder中

sparse_train，目的保留稀疏矩阵中指定的非空值，其余位置是0。
下面代码就是把c矩阵进行[True, False, True, True]的mask之后的dropout结果，结果如下:

In [14]:
a = [[0, 0], [1, 0], [2, 1], [3, 1]]
b = [1, 2, 3, 4]
shape = [4, 2]
c = tf.sparse_placeholder(tf.float32)
d = tf.sparse_retain(c, tf.convert_to_tensor([1, 0, 1, 1]))

with tf.Session() as sess:
    print(sess.run(c, feed_dict={c: (a, b, shape)}))
    print(sess.run(d, feed_dict={c: (a, b, shape)}))
    print(sess.run(tf.sparse_tensor_to_dense(d), feed_dict={c: (a, b, shape)}))

SparseTensorValue(indices=array([[0, 0],
       [1, 0],
       [2, 1],
       [3, 1]]), values=array([1., 2., 3., 4.], dtype=float32), dense_shape=array([4, 2]))
SparseTensorValue(indices=array([[0, 0],
       [2, 1],
       [3, 1]]), values=array([1., 3., 4.], dtype=float32), dense_shape=array([4, 2]))
[[1. 0.]
 [0. 0.]
 [0. 3.]
 [0. 4.]]


测试：稀疏矩阵可以乘稠密矩阵

In [15]:
a = [[0, 0], [1, 0], [1, 1], [2, 1], [3, 1]]
b = [1, 2, 2, 3, 4]
shape = [4, 2]
c = tf.sparse_placeholder(tf.float32)
d = tf.convert_to_tensor([[10.0, 1.0], [5.0, 2.0]])

with tf.Session() as sess:
    print(sess.run(tf.sparse_tensor_to_dense(c), feed_dict={c: (a, b, shape)}))
    print(sess.run(tf.sparse_tensor_dense_matmul(c, d), feed_dict={c: (a, b, shape)}))

[[1. 0.]
 [2. 2.]
 [0. 3.]
 [0. 4.]]
[[10.  1.]
 [30.  6.]
 [15.  6.]
 [20.  8.]]
